In [647]:
import pandas as pd
import numpy as np
import scipy.stats as stats

%matplotlib inline

from preprocess import impute
from sklearn.metrics import mean_squared_error
from scipy.stats import skew

import seaborn as sns
import matplotlib.pyplot as plt
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(train.shape)


(1460, 81)


In [648]:
# columns where NaN values have meaning e.g. no pool etc.

all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))


# 3 rows where PoolQC is NA, but PoolArea >0
temp = all_data[['PoolArea']][all_data["PoolQC"].isna()]
temp[temp.PoolArea > 0] 

# Everything is consistent!
temp = all_data[['Fireplaces']][all_data["FireplaceQu"].isna()]
temp[temp.Fireplaces > 0] 

# 1 where GarageCond is NA, but GarageArea >0
temp = all_data[['GarageArea']][all_data["GarageYrBlt"].isna()]
temp[temp.GarageArea > 0] 

all_data = impute(all_data)


#all_data.isnull().sum()
#list(set(train.columns.values) - set(all_data.columns.values))

In [649]:
# total = all_data.isnull().sum().sort_values(ascending=False)
# percent = (all_data.isnull().sum()/all_data.isnull().count()).sort_values(ascending=False)
# missing_data = pd.concat([total, percent], axis=1, keys=['Missing', 'Percent'])
# missing_data[missing_data.Missing > 0]


# f , ax = plt.subplots(figsize = (14,12))
# plt.title('Correlation with Sale Price',y=1,size=16)
# sns.heatmap(correlation,square = True,  vmax=0.8)

In [650]:
#list(set(all_data.columns.values))


ordinal = ['LotShape', 'Utilities', 'LandSlope', 'ExterQual','ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1', 
'BsmtFinType2', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish','GarageQual', 
'GarageCond', 'PavedDrive', 'Fence']

lot = {'Reg': 4 , 'IR1': 3, 'IR2': 2, 'IR3': 1}
util = {'AllPub': 4 , 'NoSewr': 3, 'NoSeWa': 2, 'ELO': 1}
slope = {'Gtl': 3, 'Mod': 2, 'Sev': 1}
expo = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'none': 0}
bsmt_exposure = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'none': 0}
bsmt_fin = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'none': 0}
electrical = {'SBrkr': 5, 'FuseA': 4, 'FuseF': 3, 'FuseP': 2, 'Mix': 1}
functional = {'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal': 1}
finish = {'Fin': 3, 'RFn': 2, 'Unf': 1, 'none': 0}
paved = {'Y': 2, 'P': 1, 'N': 0}
fence = {'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'none': 0}


all_data = all_data.replace({'LotShape': lot,
                         'LandSlope': slope,
                         'ExterQual': expo,
                         'ExterCond': expo,
                         'BsmtQual': expo,
                         'BsmtCond': expo,
                         'BsmtExposure': bsmt_exposure,
                         'BsmtFinType1': bsmt_fin,
                         'BsmtFinType2': bsmt_fin,
                         'HeatingQC': expo,
                         'Electrical': electrical,
                         'KitchenQual': expo,
                         'Functional': functional,
                         'FireplaceQu': expo,
                         'GarageFinish': finish,
                         'GarageQual': expo,
                         'GarageCond': expo,
                         'PavedDrive': paved,
                         'Fence': fence}) 

In [652]:
# y = train.SalePrice
# train = impute(train)
# sns.jointplot(train.TotalBsmtSF, y)


In [653]:
#log transform the target:
train["SalePrice"] = np.log1p(train["SalePrice"])

#log transform skewed numeric features:
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

all_data.drop('GarageYrBlt', axis = 1)
all_data = pd.get_dummies(all_data)


In [654]:
#all_data.loc[:, ['TotalBsmtSF', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF']]


# HOW TO IMPUTE LOTFRONTAGE? 
#all_data.loc[:, ['LotArea', 'LotFrontage', 'LotConfig']]
#sns.boxplot(all_data['LotConfig'].dropna(), all_data['LotFrontage'])

# HOW TO IMPUTE GARAGE CONDITION?
#703 
# print(len(all_data[['YearBuilt','GarageYrBlt']][all_data['GarageYrBlt'] != all_data['YearBuilt']]))
# #2216
# print(len(all_data[['YearBuilt','GarageYrBlt']][all_data['GarageYrBlt'] == all_data['YearBuilt']]))

# print(all_data[['YearBuilt','GarageYrBlt']][all_data['GarageYrBlt'] != all_data['YearBuilt']])
# print(np.mode(all_data['GarageYrBlt'].dropna() - all_data['YearBuilt'].dropna()))

#all_data[['YearBuilt','GarageYrBlt']][all_data['GarageYrBlt'].isna()]


# fig, ax = plt.subplots(figsize=(10,8))
# all_data.groupby(['GarageYrBlt','GarageCond']).agg({'GarageCond' : 'count'}).unstack().plot(ax=ax)

#all_data[['GrLivArea', 'TotRmsAbvGrd']]

In [655]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

print(X_train.shape)
print(X_test.shape)
print(y.shape)

(1460, 308)
(1459, 308)
(1460,)


In [656]:
import sklearn.feature_selection as fs
#Check the variance of all features
d = dict(zip(X_train.columns, np.var(X_train[skewed_feats])))

print('Mean variance of all numeric features = ' , np.mean(list(d.values())))
#Remove features having low variance, ie- less than a threshold 
select1 = fs.VarianceThreshold(threshold=0.073).fit(X_train)
reduced_model_fs = X_train[X_train.columns[select1.get_support(indices=True)]]


lr = LinearRegression()
# lr.fit(X_train[reduced_model_fs.columns], y)
# lr_train_pred = lr.predict(X_train[reduced_model_fs.columns])
# lr_pred = np.expm1(lr.predict(X_test[reduced_model_fs.columns]))
# print('Linear Regression using variance FS error :', rmsle(y, lr_train_pred))

Mean variance of all numeric features =  2.2765364858615693


In [658]:
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression
from sklearn.pipeline import make_pipeline

# ENet = ElasticNet(max_iter=1000000)
# grid_param = [{'alpha': np.logspace(-2, 3, 1000), 'l1_ratio': np.arange(0,1,0.1)}]
# para_search = GridSearchCV(ENet, param_grid=grid_param, cv=5, return_train_score=True)
# para_search = para_search.fit(X_train, y)

# print(para_search.best_score_)
# print(para_search.best_params_)


e = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3)
ENet = make_pipeline(RobustScaler(), e)

ENet.fit(X_train, y)
enet_train_pred = ENet.predict(X_train)
enet_pred = np.expm1(ENet.predict(X_test))
print('ENet error : ', rmsle(y, enet_train_pred))

#print(list(zip(e.coef_, X_train.columns)))

ENet error :  0.10344948199876194


In [659]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
import xgboost as xgb
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)


model_xgb.fit(X_train, y)
xgb_train_pred = model_xgb.predict(X_train)
xgb_pred = np.expm1(np.expm1(model_xgb.predict(X_test)))
print('XGBoost error : ', rmsle(y, xgb_train_pred))

from sklearn.cross_validation import cross_val_score
print('cross_val_score :', cross_val_score(model_xgb, X_train, y, cv=5))


# grid_param = [{'gamma': np.arange(0, 1, 0.05), 'max_depth' : range(0,4,1), 
#                'reg_alpha' : np.arange(0, 1, 0.02), 'reg_lambda': np.arange(0, 1, 0.05)}]
# para_search = GridSearchCV(model_xgb, param_grid=grid_param, cv=5, return_train_score=True)
# para_search = para_search.fit(X_train, y)

# print(para_search.best_score_)
# print(para_search.best_params_)


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in expm1
  from ipykernel import kernelapp as app


XGBoost error :  0.08263637106428398


In [643]:
# grid_param = [{'gamma': np.arange(0, 1, 0.05), 'max_depth' : range(0,4,1), 
#                'reg_alpha' : np.arange(0, 1, 0.02), 'reg_lambda': np.arange(0, 1, 0.05)}]
# para_search = GridSearchCV(model_xgb, param_grid=grid_param, cv=5, return_train_score=True)

model_gbr = rfr(random_state=0, 
        n_estimators=1000, max_depth=3,  max_features='auto')

model_gbr.fit(X_train, y)
gbr_train_pred = model_gbr.predict(X_train)
gbr_pred = np.expm1(model_gbr.predict(X_test))
print('Random forest error : ', rmsle(y, gbr_train_pred))

from sklearn.cross_validation import cross_val_score
print('cross_val_score :', cross_val_score(model_gbr, X_train, y, cv=5))

Random forest error :  0.013530677984518126
cross_val_score : [0.77580091 0.81327734 0.77735489 0.77900876 0.76475963]


In [644]:
# Stacking
from stacking import stacking_regression
from sklearn.ensemble import GradientBoostingRegressor as gbr, RandomForestRegressor as rfr

#models = [model_xgb, lr, rf1, rf2, gbm1, gbm2]

models = [
    # linear model, ElasticNet = lasso + ridge
    ENet,
    
    # conservative random forst model
    rfr(random_state=0,
        n_estimators=1000, max_depth=6,  max_features='sqrt'),
    
    # aggressive random forest model
    rfr(random_state=0, 
        n_estimators=1000, max_depth=9,  max_features='auto'),
    
    # conservative gbm model
    gbr(random_state=0, learning_rate = 0.005, max_features='sqrt',
        min_samples_leaf=15, min_samples_split=10, 
        n_estimators=3000, max_depth=3),
    
    # aggressive gbm model
    gbr(random_state = 0, learning_rate = 0.01, max_features='sqrt',
        min_samples_leaf=10, min_samples_split=5, 
        n_estimators = 1000, max_depth = 9)
    ]

#meta_model = lr(normalize=True)
meta_model = LinearRegression(normalize=True)
stacking_prediction = stacking_regression(models, meta_model, X_train, y, X_test,
                                           metric=rmsle, verbose=1)

stacking_prediction = np.expm1(stacking_prediction)

metric: [rmsle]

model 0: [Pipeline]
    ----
    MEAN:   [0.01106772]

model 1: [RandomForestRegressor]


/Users/lavanya/Desktop/NYC Bootcamp 2018/Course Material/Kaggle Housing Project/stacking.py:112: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X_train = X_train.as_matrix()
/Users/lavanya/Desktop/NYC Bootcamp 2018/Course Material/Kaggle Housing Project/stacking.py:113: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X_test = X_test.as_matrix()


    ----
    MEAN:   [0.01275823]

model 2: [RandomForestRegressor]
    ----
    MEAN:   [0.01136467]

model 3: [GradientBoostingRegressor]
    ----
    MEAN:   [0.00997297]

model 4: [GradientBoostingRegressor]
    ----
    MEAN:   [0.00992767]



In [480]:
ensembling_train = pd.DataFrame({'enet': enet_train_pred,
     'xgb': xgb_train_pred
    })


print(ensembling_train.shape)
print(y.shape)
model_ensembled = LinearRegression().fit(ensembling_train, y)
ensembled_train_pred = model_ensembled.predict(ensembling_train)
#ensembled_pred = np.expm1(model_ensembled.predict(X_test))
print('Ensembling error : ', rmsle(y, ensembled_train_pred))

from sklearn.cross_validation import cross_val_score
print('cross_val_score :', cross_val_score(model_ensembled, ensembling_train, y, cv=5))

print(model_ensembled.coef_)
ensembled_pred = (0.33 * enet_pred) + (0.66 * xgb_pred)
ensembled_pred


(1460, 2)
(1460,)
Ensembling error :  0.08188460340162537
cross_val_score : [0.95677779 0.96273976 0.95442713 0.9615877  0.95225991]
[-0.22226586  1.23130504]


ValueError: shapes (1460,292) and (2,) not aligned: 292 (dim 1) != 2 (dim 0)

In [510]:
import csv
csv_file = open('submission1.csv', 'w')
writer = csv.writer(csv_file)

Id = np.arange(1461, 2920)
writer.writerow(['Id', 'SalePrice'])


dictionary = dict(zip(Id, gbr_pred))

for key, value in dictionary.items():
    writer.writerow([key, value])
    csv_file.flush()